In [3]:
import pandas as pd
import numpy as np

class FeatureEngineering:
    def __init__(self, df, wet_threshold=200):
        self.df = df.copy()
        self.wet_threshold = wet_threshold

    def preprocess(self):
        # تحويل الوقت لـ datetime
        self.df['time'] = pd.to_datetime(self.df['time'], format='%H:%M:%S', errors='coerce')
        
        # تحويل state لـ 0/1
        self.df['state'] = self.df['state'].map({'dry':0, 'wet':1})
        
        # تحويل soil_type لأرقام
        self.df['soil_type'] = self.df['soil_type'].astype('category').cat.codes
        
        # ملأ القيم الناقصة
        self.df['moisture_percent'] = self.df['moisture_percent'].fillna(method='ffill').fillna(0)
        self.df['time'] = self.df['time'].fillna(method='ffill')
        self.df['pump_state'] = self.df['pump_state'].fillna(0)
        self.df['state'] = self.df['state'].fillna(0)
        self.df['soil_type'] = self.df['soil_type'].fillna(0)
        
        return self.df

    def compute_time_to_dry(self):
        self.df['time_to_dry_sec'] = 0.0
        last_slope = -0.5
        
        for i in range(len(self.df)):
            window = self.df.iloc[max(0, i-5):i+1]
            window = window[window['pump_state']==0]
            
            if len(window) >= 2:
                x = (window['time'] - window['time'].iloc[0]).dt.total_seconds()
                y = window['moisture_percent']
                slope = (y.iloc[-1] - y.iloc[0]) / (x.iloc[-1] - x.iloc[0])
                
                if abs(slope) < 0.01:
                    slope = last_slope
                else:
                    last_slope = slope
                
                seconds_to_dry = abs((1000 - y.iloc[-1]) / slope)
                self.df.at[i, 'time_to_dry_sec'] = seconds_to_dry
        
        # ملأ أي قيم ناقصة بالـ ffill أو 0
        self.df['time_to_dry_sec'] = self.df['time_to_dry_sec'].fillna(method='ffill').fillna(0)
        
        return self.df

    def get_features_and_target(self):
        self.preprocess()
        self.compute_time_to_dry()
        
        feature_cols = ['moisture_percent', 'pump_state', 'soil_type', 'state']
        X = self.df[feature_cols].astype(float)
        y = self.df['time_to_dry_sec'].astype(float)
        
        return X, y


In [18]:
df = pd.read_csv(r"D:\Smart Irregation System\DataCollection\Collecteddata.csv")
fe = FeatureEngineering(df)
X, y = fe.get_features_and_target()

print(X.head())
print(y.head())

   moisture_percent  pump_state  soil_type  state
0            1000.0         0.0        0.0    1.0
1             950.0         1.0        0.0    1.0
2             235.0         0.0        0.0    1.0
3             234.0         0.0        0.0    1.0
4             238.0         0.0        0.0    1.0
0      0.0
1      0.0
2    120.0
3    180.0
4    240.0
Name: time_to_dry_sec, dtype: float64


C:\Users\AdminOS\AppData\Local\Temp\ipykernel_7300\1243848178.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['moisture_percent'] = self.df['moisture_percent'].fillna(method='ffill').fillna(0)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_7300\1243848178.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time'] = self.df['time'].fillna(method='ffill')
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_7300\1243848178.py:50: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time_to_dry_sec'] = self.df['time_to_dry_sec'].fillna(method='ffill').fillna(0)


In [9]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df = pd.read_csv(r"D:\Smart Irregation System\DataCollection\Collecteddata.csv")
fe = FeatureEngineering(df)

fe = FeatureEngineering(df)
X, y = fe.get_features_and_target()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# ---------------------------

# ---------------------------
# 3. تدريب موديل XGBoost
# ---------------------------
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)
r2 = r2_score(y_val, y_pred)
print(f"R² Score on validation set: {r2:.4f}")

# ---------------------------
# 4. التقييم
# ---------------------------
y_pred = model.predict(X_val)
r2 = r2_score(y_val, y_pred)
print(f"R² Score on validation set: {r2:.4f}")

C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['moisture_percent'] = self.df['moisture_percent'].fillna(method='ffill').fillna(0)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time'] = self.df['time'].fillna(method='ffill')
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:50: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time_to_dry_sec'] = self.df['time_to_dry_sec'].fillna(method='ffill').fillna(0)


R² Score on validation set: 0.5430
R² Score on validation set: 0.5430


In [13]:
import joblib

# حفظ الموديل في ملف
joblib.dump(model, r"D:\Smart Irregation System\Model\xgb_model.pkl")

print("Model saved successfully!")


Model saved successfully!


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# ---------------------------
# 1. تجهيز البيانات
# ---------------------------
df = pd.read_csv(r"D:\Smart Irregation System\DataCollection\Collecteddata.csv")
fe = FeatureEngineering(df)
X, y = fe.get_features_and_target()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------------------------
# 2. تدريب موديل Random Forest
# ---------------------------
rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=10,   # تقدر تعدل
    random_state=42
)

rf_model.fit(X_train, y_train)

# ---------------------------
# 3. التقييم
# ---------------------------
y_pred = rf_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
print(f"R² Score on validation set (RF): {r2:.4f}")



C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['moisture_percent'] = self.df['moisture_percent'].fillna(method='ffill').fillna(0)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time'] = self.df['time'].fillna(method='ffill')
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:50: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time_to_dry_sec'] = self.df['time_to_dry_sec'].fillna(method='ffill').fillna(0)


R² Score on validation set (RF): 0.5430


In [12]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# ---------------------------
# 1. تجهيز البيانات
# ---------------------------
df = pd.read_csv(r"D:\Smart Irregation System\DataCollection\Collecteddata.csv")
fe = FeatureEngineering(df)
X, y = fe.get_features_and_target()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------------------------
# 2. تدريب موديل KNN Regressor
# ---------------------------
knn_model = KNeighborsRegressor(
    n_neighbors=5,   # ممكن تغييره للتجربة
    weights='uniform',  # أو 'distance'
    p=2  # Euclidean distance
)
knn_model.fit(X_train, y_train)

# ---------------------------
# 3. التقييم
# ---------------------------
y_pred = knn_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
print(f"R² Score on validation set (KNN Regressor): {r2:.4f}")


R² Score on validation set (KNN Regressor): 0.4442


C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['moisture_percent'] = self.df['moisture_percent'].fillna(method='ffill').fillna(0)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time'] = self.df['time'].fillna(method='ffill')
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_25364\1243848178.py:50: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df['time_to_dry_sec'] = self.df['time_to_dry_sec'].fillna(method='ffill').fillna(0)
